In [1]:
#pip install rusenttokenize

In [2]:
import re
import pandas as pd
import numpy as np
import html
import json
import warnings
from bs4 import BeautifulSoup
from rusenttokenize import ru_sent_tokenize
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [3]:
data_match_news = pd.read_csv("C:/Users/Alexey/Master's dissertation/ods_match_news.csv")
data_broadcasts = pd.read_csv("C:/Users/Alexey/Master's dissertation/ods_broadcasts_201905301157.csv")

In [4]:
#Изменение типов столбцов у датафреймов
data_match_news = data_match_news.astype({'body': np.str})
data_broadcasts = data_broadcasts.astype({'content': np.str})

In [5]:
#Предобработка данных
#Удаление \n \r \t. Удаление html разметки и замена мнемоник html

regex = re.compile(r'[\n\r\t]')
regex_del_char = re.compile('[^]a-zA-Zа-яА-Я0-9 "#$%&\'\(\)\*\+\,-\./:;<=>\?@\[\\^_!`\{|\}]')

data_match_news['name'] = data_match_news['name'].apply(lambda x: BeautifulSoup(html.unescape(regex.sub(" ", x)), 'lxml').text)
data_match_news['body'] = data_match_news['body'].apply(lambda x: BeautifulSoup(html.unescape(regex_del_char.sub("", regex.sub(" ", x))), 'lxml').text)

data_broadcasts['content'] = data_broadcasts['content'].apply(lambda x: BeautifulSoup(html.unescape(regex_del_char.sub("", regex.sub(" ", x))), 'lxml').text)

In [6]:
data_match_news

,id,name,ctime,body,match_id
0,7005921,Зико: «Задача может быть только одна: пройти в...,2009-02-18 01:06:26,Главный тренер ЦСКА Зико поделился мыслями о п...,18936
1,7005943,"Текстовые онлайны матчей «Зенит» – «Штутгарт»,...",2009-02-18 01:08:20,Сегодня на Sports.ru дебютируют текстовые онла...,18935
2,7005943,"Текстовые онлайны матчей «Зенит» – «Штутгарт»,...",2009-02-18 01:08:20,Сегодня на Sports.ru дебютируют текстовые онла...,18936
3,7006022,Стилян Петров: «ЦСКА выиграл все матчи в этом ...,2009-02-18 01:21:55,Полузащитник «Астон Виллы» Стилян Петров подел...,18936
4,7006398,Хиддинк постетит матч «Астон Виллы» и ЦСКА,2009-02-18 03:17:23,Главный тренер сборной России и «Челси» Гус Хи...,18936
...,...,...,...,...,...
92992,1040645221,Петр Чех: Если бы пропустил от Денисова — это ...,2016-06-02 14:44:24,Голкипер сборной Чехии Петр Чех после победы в...,1095139
92993,1040648050,"Конор Шери: «Кросби сказал, что выиграет вбрас...",2016-06-02 16:17:07,Форвард Питтсбурга Конор Шери прокомментировал...,1104866
92994,1040648436,Сидни Кросби: «Победным голом мы больше обязан...,2016-06-02 16:29:35,Капитан Питтсбурга Сидни Кросби прокомментиров...,1104866
92995,1040653602,Джастин Браун: «Питтсбург» все наступал и наст...,2016-06-02 19:36:29,Защитник Сан-Хосе Джастин Браун оценил ход сер...,1104866


In [7]:
data_broadcasts

,match_id,team1,team2,name,match_time,type,minute,content,message_time
0,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"Добрый день! Sports.ru поздравляет всех, кто п...",1237034763
1,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"Будем надеяться, что признаков пресловутого ""в...",1237034996
2,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"Что ж, сегодняшних соперников можно назвать од...",1237035202
3,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,Зимой подмосковный клуб едва не исчез с футбол...,1237035415
4,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"""Локомотив"" зимой старался стать лучше. В сост...",1237035796
...,...,...,...,...,...,...,...,...,...
722062,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,"2:1 - ""Питтсбург"" выигрывает второй финальный ...",1464837758
722063,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,"Очевидно, что залог успеха ""Питтсбурга"" - иниц...",1464837879
722064,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,"Другими словами, ДеБур должен в первую очередь...",1464838029
722065,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,Большое спасибо за чтение! До новых встреч на ...,1464838220


In [8]:
data_all_name_news = data_match_news.sort_values(['match_id', 'ctime'], ascending = True).groupby(['match_id']).agg(count_sentences=('match_id', 'count'), news=('body', 'first')).reset_index()
data_all_name_news.values[1]

array([18935, 33,
       'Сегодня на Sports.ru дебютируют текстовые онлайны матчей Кубка УЕФА, кроме того, на сайте продолжится тестирование трансляций чемпионата Англии. За всеми значимыми событиями (голы, замены, предупреждения и т.д.) всех матчей дня вы сможете следить на страницах онлайнов, обмениваясь по ходу впечатлениями в матч-чате. Попасть на страницы текстовых трансляций вы можете через страницу матч-центра либо с помощью информационного блока непосредственно под главным меню. При этом для трех матчей предусмотрены подробнейшие текстовые трансляции: это Зенит  Штутгарт (онлайн начинается в 19.45), Астон Вилла  ЦСКА (начало  в 21.30) и Манчестер Юнайтед  Фулхэм (начало  в 22.30). Заранее приносим извинения за возможные на этапе бета-тестирования ошибки и призываем тех, кто их заметил, писать на  support@sports.ru.'],
      dtype=object)

In [9]:
data_all_text_broadcasts = data_broadcasts.sort_values(['match_id','message_time'], ascending = True).groupby(['match_id'])['content'].apply(lambda x: ' '.join(x).lower().strip()).reset_index()
data_all_text_broadcasts

,match_id,content
0,18925,"добрый день! sports.ru поздравляет всех, кто п..."
1,18935,"матчем со ""штутгартом"" ""зенит"" открывает европ..."
2,18936,"интересный парадокс жребия - армейцам, единств..."
3,18941,приветствуем всех любителей самой популярной и...
4,18956,"ответный матч 1/16 финала кубка уефа ""зенита"" ..."
...,...,...
8776,1104438,привет. теперь точно сегодня последний тур рф...
8777,1104439,всем привет! стыковые матчи за право выступать...
8778,1104440,всем привет! сегодня важный день для российско...
8779,1104865,"привет всем, кто не представляет своей жизни б..."


In [10]:
prepared_data = data_all_text_broadcasts.merge(data_all_name_news[['match_id','news']].set_index('match_id'), on='match_id')
prepared_data

,match_id,content,news
0,18925,"добрый день! sports.ru поздравляет всех, кто п...",Более трех тысяч сотрудников столичной милиции...
1,18935,"матчем со ""штутгартом"" ""зенит"" открывает европ...",Сегодня на Sports.ru дебютируют текстовые онла...
2,18936,"интересный парадокс жребия - армейцам, единств...",Главный тренер ЦСКА Зико поделился мыслями о п...
3,18941,приветствуем всех любителей самой популярной и...,Главный тренер «Тоттенхэма» Харри Реднапп в пр...
4,18956,"ответный матч 1/16 финала кубка уефа ""зенита"" ...",На предматчевой пресс-конференции перед завтра...
...,...,...,...
7666,1104438,привет. теперь точно сегодня последний тур рф...,В ответных стыковых матчах за право выступать ...
7667,1104439,всем привет! стыковые матчи за право выступать...,Информация о возможном объединении московского...
7668,1104440,всем привет! сегодня важный день для российско...,В ответных стыковых матчах за право выступать ...
7669,1104865,"привет всем, кто не представляет своей жизни б...",В ночь с понедельника на вторник состоится пер...


In [11]:
train, test = train_test_split(prepared_data, test_size=0.1, random_state=1234)

In [12]:
with open('my_data_train.txt', 'w', encoding='utf-8') as outfile:
    for i, row in train.iterrows():
        data_json = {'text': ru_sent_tokenize(row['content']), 'summary': ru_sent_tokenize(row['news'])}
        json.dump(data_json, outfile, ensure_ascii=False)
        outfile.write('\n')

In [13]:
with open('my_data_test.txt', 'w', encoding='utf-8') as outfile:
    for i, row in test.iterrows():
        data_json = {'text': ru_sent_tokenize(row['content']), 'summary': ru_sent_tokenize(row['news'])}
        json.dump(data_json, outfile, ensure_ascii=False)

In [14]:
data_match_news.to_csv("C:/Users/Alexey/Master's dissertation/data_match_news_prep.csv")
data_broadcasts.to_csv("C:/Users/Alexey/Master's dissertation/data_broadcasts_prep.csv")